# Import libraries

In [1]:
import os
import math
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from copy import deepcopy
from keras.layers import *
from keras.layers import LSTM
from keras.layers import Dense
from datetime import datetime
from keras.layers import Dropout
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error,r2_score

# Loading Data

In [4]:
#df=pd.read_csv('../MPT Data/Combined.csv')
df=pd.read_csv('../MPT Data/Combined.csv')
df.drop(['TRADE_ENTRY_DATE','Unnamed: 0'],axis=1,inplace=True)
for _,columns in enumerate(df):
    df[[columns]]=df[[columns]].astype('float')
df.columns=['ATRL','bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML',
      'OGDC','PPL','PSO','SEARL','TRG']
df.head()

,ATRL,bahl,EPCL,FCCL,LUCK,MCB,MLCF,NETSOL,NML,OGDC,PPL,PSO,SEARL,TRG
0,82.90,26.50,22.75,6.70,51.35,144.00,5.00,17.40,31.01,73.85,176.75,200.80,64.05,1.37
1,83.30,26.58,23.00,6.70,51.25,144.17,4.98,17.45,31.30,74.00,177.00,200.99,64.00,1.40
2,85.00,26.90,22.90,6.75,51.50,144.49,5.05,17.40,31.20,73.70,177.30,201.03,63.10,1.38
3,85.40,27.02,23.19,6.70,51.85,145.00,5.08,17.50,31.05,73.80,177.33,201.12,60.80,1.31
4,86.85,28.10,23.11,6.74,51.85,145.00,5.17,17.26,30.80,73.70,176.80,201.45,62.00,1.40


# Trading Framework

## Trader function

## Portfolio declaration

In [6]:
lm=250
X=2000
tpf=98000
TN=X+tpf
df1=df
shares1={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
Min={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
Max={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
for share in shares1:
    shares1[share]=math.floor(7000/df1.loc[0,share])

## Testing Framework

In [ ]:
TN=[100000]
df2=df
for i in range(df2.shape[0]//lm-1):
    df1=df2[i*lm:(i*lm)+lm]
    for share in shares1:
        Min[share]=df1[share].idxmin()
        Max[share]=df1[share].idxmax()
    for share in shares1:
        if Min[share]!=Max[share]:
           X,shares1[share]=trader(X,shares1[share],df2.loc[Min[share],share],df2.loc[Max[share],share])
           tpf+=(df2.loc[(i*lm)+lm,share]*shares1[share])
           TN.append(X+tpf)
           print(X,tpf)
    tpf=0
            
    

# Preprocessing

## Train Test split

In [7]:
train_data=[]
test_data=[]
for share in shares1:
    train_data.append(df.loc[:299999,share])
    test_data.append(df.loc[300000:400000,share])


### Normalizing Data
Now you need to define a scaler to normalize the data. MinMaxScalar scales all the data to be in the region of 0 and 1. You can also reshape the training and test data to be in the shape [data_size, num_features]

In [8]:
scaler = MinMaxScaler()
for i in range(len(shares1)):
    train_data[i] = train_data[i].values.reshape(-1,1)
    test_data[i] = test_data[i].values.reshape(-1,1)
td=train_data.copy()
tr=test_data.copy()

Due to the observation we made earlier, that is, different time periods of data have different value ranges, you normalize the data by splitting the full series into windows. If you don't do this, the earlier data will be close to 0 and will not add much value to the learning process. Here you choose a window size of 25000.


In [103]:
# Train the Scaler with training data and smooth data
t_norm=[]
train_data_norm=[]
test_data_norm=[]
smoothing_window_size = 200
test_scalers,train_scalers=[],[]
for i in range(len(shares1)):
    t_norm=[]
    train_data_acq=td[i].copy() 
    test_data_acq=tr[i].copy()
    for di in range(0,300000,smoothing_window_size):
        scaler=MinMaxScaler()
        scaler.fit(train_data_acq[di:di+smoothing_window_size,:])
        train_data_acq[di:di+smoothing_window_size,:] = scaler.transform(train_data_acq[di:di+smoothing_window_size,:])
        t_norm.append(deepcopy(scaler))
    train_data_norm.append(train_data_acq)
    train_scalers.append(t_norm.copy())
#Normalize the test data too
    sc=MinMaxScaler()
    test_data_norm.append(sc.fit(test_data_acq).transform(test_data_acq).reshape(-1))
    test_scalers.append(deepcopy(sc))
# Reshape both train and test data
for i in range(len(shares1)):
    train_data_norm[i] = train_data_norm[i].reshape(-1)

## Preparing Train and test data for training

### Train data

#### Normalized Train data

In [10]:
Xt=[]
yt=[]
X_train_norm = []
y_train_norm = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(train_data_norm[j])):
        Xt.append(train_data_norm[j][i-60:i])
        yt.append(train_data_norm[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_train_norm.append(Xt)
    y_train_norm.append(yt)

###  Non Normalized Test Data

In [11]:
Xt=[]
yt=[]
X_train_comb = []
y_train_comb = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(train_data_norm[j])):
        Xt.append(train_data[j][i-60:i])
        yt.append(train_data[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_train_comb.append(Xt)
    y_train_comb.append(yt)

### Test data

#### Non Normalized Test Data

In [12]:
Xt=[]
yt=[]
X_test_comb = []
y_test_comb = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(test_data[j])):
        Xt.append(test_data[j][i-60:i])
        yt.append(test_data[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_test_comb.append(Xt)
    y_test_comb.append(yt)

### Normalized Test Data

In [13]:
Xt=[]
yt=[]
X_test_norm = []
y_test_norm = [] 
for j in range(len(shares1)):
    Xt,yt=[],[]
    for i in range(60, len(test_data[j])):
        Xt.append(test_data_norm[j][i-60:i])
        yt.append(test_data_norm[j][i])
    
    Xt, yt = np.array(Xt), np.array(yt)
    Xt = np.reshape(Xt, (Xt.shape[0], Xt.shape[1], 1))
    X_test_norm.append(Xt)
    y_test_norm.append(yt)

# Machine Learning Model 

## Model Declaration

In [14]:
def model():
    regressor = Sequential()

    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (60, 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    
    return regressor

## Storing initialization of 14 models for 14 stocks and MAPE

In [17]:
models=[]
MAPE={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDX': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
MAP=['ATRL','bahl','EPCL','FCCL','LUCK','MCB','MLCF','NETSOL','NML','OGDX',
       'PPL','PSO','SEARL','TRG']
for share in shares1:
    models.append(model()) 

## Training

In [141]:
for i in range(14):
    models[i].fit(X_train_norm[i],y_train_norm[i],epochs=100,batch_size=10000) 

Epoch 1/100
30/30 [==============================] - 10s 322ms/step - loss: 0.0074
Epoch 2/100
30/30 [==============================] - 10s 326ms/step - loss: 0.0074
Epoch 3/100
30/30 [==============================] - 10s 331ms/step - loss: 0.0074
Epoch 4/100
30/30 [==============================] - 10s 330ms/step - loss: 0.0074
Epoch 5/100
30/30 [==============================] - 10s 327ms/step - loss: 0.0074
Epoch 6/100
30/30 [==============================] - 10s 325ms/step - loss: 0.0073
Epoch 7/100
30/30 [==============================] - 10s 323ms/step - loss: 0.0074
Epoch 8/100
30/30 [==============================] - 10s 323ms/step - loss: 0.0073
Epoch 9/100
30/30 [==============================] - 10s 325ms/step - loss: 0.0073
Epoch 10/100
30/30 [==============================] - 10s 328ms/step - loss: 0.0073
Epoch 11/100
30/30 [==============================] - 10s 325ms/step - loss: 0.0073
Epoch 12/100
30/30 [==============================] - 10s 327ms/step - loss: 0.0073
E

## Saving Model Weights

In [22]:
for i in range(14):
  models[i].save(MAP[i]+'_model')


KeyboardInterrupt: 

## Loading Models

In [23]:
for i in range(14):
  print('loaded model :',MAP[i])
  models[i]=keras.models.load_model('Models/'+MAP[i]+'_model')

loaded model : ATRL
loaded model : bahl
loaded model : EPCL
loaded model : FCCL
loaded model : LUCK
loaded model : MCB
loaded model : MLCF
loaded model : NETSOL
loaded model : NML
loaded model : OGDX
loaded model : PPL
loaded model : PSO
loaded model : SEARL
loaded model : TRG


# MAPE for LSTM models

## MAPE Calculator

In [25]:
def mape(yhat,y_test):  
  arr=np.array([[]])
  for i in range(len(yhat)):
      arr1=np.array([np.abs((y_test_comb[0][i] - yhat[i]) / yhat[i])]) 
      arr=np.concatenate((arr,arr1),axis=1)
  return arr.mean()


## Calculating MAPE

In [28]:
yhat_for_all_models=[]
for i in range(14):
  yhat=models[i].predict(X_test_norm[i],batch_size=5000)
  yhat_for_all_models.append(test_scalers[i].inverse_transform(yhat))
  print('Done :',i)

Done : 0
Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13


## Saving MAPE

In [36]:
for i in range(14):
    List=[]
    List.append(mape(yhat_for_all_models[i],y_test_comb[i]))
    MAPE[MAP[i]]=List
df_MAPE=pd.DataFrame(MAPE)
df_MAPE.to_csv('MAPE.csv')

# Putting Model in Trading Framework

## Getting Prediction for train data

In [107]:
yhat_for_train=[]

for i in range(14):
  yt=[]
  yhat=models[i].predict(X_train_norm[i],batch_size=5000)
  for j in range(len(train_scalers[i])):
      yt.extend(train_scalers[i][j].inverse_transform(yhat[200*j:(200*j)+200]))
  yhat_for_train.append(yt)
  print('Done :',i) 

Done : 0
Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13


In [124]:
df_train=pd.DataFrame(yhat_for_train[0])
for i in range(1,14):
    df_temp=pd.DataFrame(yhat_for_train[i])
    df_train=pd.concat([df_train,df_temp],axis=1)
df_train

,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,85.495445,28.712223,24.044535,7.060798,54.012886,143.909180,5.060462,17.288521,31.123518,73.995895,173.974640,199.638260,49.915535,1.642860
1,85.276649,28.628889,23.997244,7.035091,53.624226,143.681442,5.099741,17.318390,31.065128,74.010178,173.765945,199.617630,50.199841,1.642868
2,85.119873,28.738647,23.855869,7.031153,53.951996,143.649918,5.077675,17.355007,31.153982,73.996681,173.860306,199.661713,50.309097,1.642910
3,85.089920,29.171711,23.800381,7.073793,54.362797,143.720032,5.083728,17.424330,31.173611,74.114754,174.182144,199.311737,51.082546,1.606955
4,85.004517,29.383968,23.788126,7.097402,54.425026,143.923843,5.053038,17.501820,31.224255,74.345009,174.749466,199.111786,50.506191,1.596442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299935,217.625076,84.500397,33.630356,17.507677,433.524689,190.203186,31.509790,76.900787,170.038574,175.923737,110.916931,357.105408,210.007874,42.312984
299936,217.695312,84.500397,33.630356,17.507677,433.485229,190.169022,31.509790,76.900787,169.912079,175.921112,110.839447,357.100494,210.007874,42.312408
299937,217.541809,84.500397,33.630356,17.507677,433.448944,190.166946,31.509790,76.900787,169.866364,175.929886,110.752228,357.098480,210.007874,42.312199
299938,217.508057,84.500397,33.630356,17.507677,433.416718,190.158936,31.509790,76.900787,169.646072,175.923080,110.967163,356.789459,210.007874,42.253231


## Getting Prediction for test data

In [ ]:
yhat_for_all_models=[]
for i in range(14):
  yhat=models[i].predict(X_test_norm[i],batch_size=5000)
  yhat_for_all_models.append(test_scalers[i].inverse_transform(yhat))
  print('Done :',i)

In [128]:
df1=pd.DataFrame(yhat_for_all_models[0])
for i in range(1,14):
    df2=pd.DataFrame(yhat_for_all_models[i])
    df1=pd.concat([df1,df2],axis=1)
df1

,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,214.642029,84.500397,33.630356,17.507677,442.986328,187.670898,31.50979,76.900787,168.380676,178.574905,114.264664,355.441254,210.007874,42.569389
1,214.823959,84.500397,33.630356,17.507677,443.152527,187.583374,31.50979,76.900787,168.345428,178.597610,114.267494,355.517883,210.007874,42.420624
2,214.625107,84.500397,33.630356,17.507677,442.524475,187.519211,31.50979,76.900787,168.252106,178.596573,114.304619,355.537781,210.007874,42.353878
3,214.978348,84.500397,33.630356,17.507677,442.334900,187.508179,31.50979,76.900787,167.905991,178.599899,114.379982,355.753235,210.007874,42.348030
4,215.056732,84.500397,33.630356,17.507677,442.359711,187.507431,31.50979,76.900787,168.016739,178.845871,114.389366,355.979828,210.007874,42.328754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99936,368.681671,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.539398,210.007874,35.086452
99937,368.380188,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.575470,210.007874,35.087753
99938,368.286804,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.612488,210.007874,35.087685
99939,368.123657,84.500397,33.630356,17.507677,400.109039,189.229401,31.50979,76.900787,126.546799,145.979904,176.305267,478.741150,210.007874,35.087456


## Merging train and test set

In [168]:
df_pred=pd.concat([df_train,df1])
df_pred.columns=MAP
df_pred.reset_index(inplace=True)
df_pred.drop({'index'},axis=1,inplace=True)

## Saving the predictions

In [170]:
df_pred.to_csv('Predicted_values.csv',index=None)

## Testing Predictions

### Loading Files

In [171]:
df_pred=pd.read_csv('Predicted_values.csv',index_col=None)
#df_pred.drop({'index','Unnamed:0'},axis=1,inplace=True)
df_pred

,ATRL,bahl,EPCL,FCCL,LUCK,MCB,MLCF,NETSOL,NML,OGDX,PPL,PSO,SEARL,TRG
0,85.495445,28.712223,24.044535,7.060798,54.012886,143.909180,5.060462,17.288521,31.123518,73.995895,173.974640,199.638260,49.915535,1.642860
1,85.276649,28.628889,23.997244,7.035091,53.624226,143.681442,5.099741,17.318390,31.065128,74.010178,173.765945,199.617630,50.199841,1.642868
2,85.119873,28.738647,23.855869,7.031153,53.951996,143.649918,5.077675,17.355007,31.153982,73.996681,173.860306,199.661713,50.309097,1.642910
3,85.089920,29.171711,23.800381,7.073793,54.362797,143.720032,5.083728,17.424330,31.173611,74.114754,174.182144,199.311737,51.082546,1.606955
4,85.004517,29.383968,23.788126,7.097402,54.425026,143.923843,5.053038,17.501820,31.224255,74.345009,174.749466,199.111786,50.506191,1.596442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399876,368.681671,84.500397,33.630356,17.507677,400.109039,189.229401,31.509790,76.900787,126.546799,145.979904,176.305267,478.539398,210.007874,35.086452
399877,368.380188,84.500397,33.630356,17.507677,400.109039,189.229401,31.509790,76.900787,126.546799,145.979904,176.305267,478.575470,210.007874,35.087753
399878,368.286804,84.500397,33.630356,17.507677,400.109039,189.229401,31.509790,76.900787,126.546799,145.979904,176.305267,478.612488,210.007874,35.087685
399879,368.123657,84.500397,33.630356,17.507677,400.109039,189.229401,31.509790,76.900787,126.546799,145.979904,176.305267,478.741150,210.007874,35.087456


### Trader Function

In [193]:
def trader(X,nS3,share_price1,share_price2):
    '''
    This function takes cash,Number of shares
    Share price at minimum and share price at maximum
    and returns the remaining amount
    
    We are selling 20pc of stock at maximum price and buy
    it at minimum point
    '''
    #Selling shares at maximum point in 100 entries
    Amount_Gained=((nS3*share_price2)*.2)
    Brocker_fee=((nS3*share_price2)*.2)*.3*.01
    nS3=nS3-(nS3*.2)
    X=X+Amount_Gained-Brocker_fee
    #Buying shares at minimum point in 100 entries
    Shares_bought=(.997*Amount_Gained)/share_price1
    nS3=nS3+Shares_bought
    X=X-(.997*Amount_Gained)
    return X,math.floor(nS3)

### Putting in our trading framework

In [188]:
lm=1000 #SLiding window size
X=2000
tpf=98000
TN=X+tpf
df1=df
shares1={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
Min={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
Max={'ATRL': 0,'bahl': 0,'EPCL': 0,'FCCL': 0,'LUCK': 0,'MCB': 0,
 'MLCF': 0,'NETSOL': 0,'NML': 0,'OGDC': 0,'PPL': 0,'PSO': 0,
 'SEARL': 0,'TRG': 0}
for share in shares1:
    shares1[share]=math.floor(7000/df1.loc[0,share])

### Simulating as per predicted data

In [189]:
TN=[100000]
df2=df
for i in range(df2.shape[0]//lm-1):
    df1=df2[i*lm:(i*lm)+lm]
    for share in shares1:
        Min[share]=df1[share].idxmin()
        Max[share]=df1[share].idxmax()
    for share in shares1:
        if Min[share]!=Max[share]:
           X,shares1[share]=trader(X,shares1[share],df2.loc[Min[share],share],df2.loc[Max[share],share])
           tpf+=(df2.loc[(i*lm)+lm,share]*shares1[share])
           TN.append(X+tpf)
           print(X,tpf)
    tpf=0

2000.0 106856.6
2000.0 116026.6
2000.0 123602.90000000001
2000.0000000000002 131235.05000000002
2000.0 139428.45
2000.0000000000002 147047.46000000002
2000.0000000000002 155145.21000000002
2000.0000000000005 163751.83000000002
2000.0000000000005 172433.23
2000.0000000000005 179946.78
2000.0000000000007 187047.12
2000.0000000000007 193855.62
2000.0000000000007 201542.38
2000.0000000000005 213448.38
2000.0 12070.4
2000.000000000001 24181.12
2000.0000000000011 31388.839999999997
2000.000000000001 39202.03999999999
2000.000000000001 47887.53999999999
2000.0000000000007 56016.63999999999
2000.0000000000002 63328.63999999999
2000.0000000000002 69745.81999999999
2000.0000000000005 79389.51999999999
2000.0000000000007 87118.48
2000.000000000001 94604.87999999999
2000.0000000000011 101849.87999999999
2000.0000000000014 109835.87999999999
2000.0000000000018 119583.45999999999
2000.0000000000014 11209.5
2000.0000000000018 22063.5
2000.0000000000016 28953.9
2000.000000000002 38721.9
2000.000000000

2000.0000000000146 18884.800000000003
2000.000000000015 37747.3
2000.0000000000136 73391.3
2000.000000000014 91642.1
2000.0000000000146 104620.1
2000.0000000000218 419295.19999999995
2000.0000000000227 457423.1
2000.0000000000218 492314.83999999997
2000.0000000000223 507085.81999999995
2000.000000000022 514493.3599999999
2000.0000000000223 526696.46
2000.0000000000218 795912.46
2000.000000000029 1306348.2599999998
2000.000000000029 17353.0
2000.000000000029 39042.1
2000.000000000029 74827.29999999999
2000.0000000000286 94319.79999999999
2000.0000000000286 107751.4
2000.000000000029 423832.91000000003
2000.00000000003 458786.63
2000.000000000031 493300.55
2000.0000000000305 508036.95
2000.0000000000302 515680.95
2000.000000000031 528311.1
2000.000000000029 794753.1
2000.000000000029 1383453.85
2000.000000000029 19068.4
2000.000000000029 39702.28
2000.0000000000282 76981.08
2000.0000000000286 97112.28
2000.0000000000282 112174.04
2000.0000000000218 406082.54
2000.000000000021 435927.5
20

2000.0 26745.25
2000.0 136750.95
2000.0 281493.03
2000.0 308561.63
2000.0000000000005 324257.63
2000.0 1192542.23
2000.0 1652718.8699999999
2000.0 1695227.67
2000.0000000000005 1714110.8699999999
2000.0000000000002 1722657.8699999999
2000.0 1733780.67
2000.0 2404871.25
2000.0 9914302.05
2000.0 26115.75
2000.0 155683.34999999998
2000.0 305608.94999999995
2000.0 331970.76999999996
2000.0 347447.17
2000.0 1199185.3699999999
2000.0 1633289.8699999999
2000.0 1676299.2699999998
2000.0000000000005 1694971.2699999998
2000.0000000000005 1703626.63
2000.000000000001 1714749.43
2000.0 2414731.99
2000.0 9593775.190000001
2000.0 24730.800000000003
2000.0 153973.04
2000.0 295576.24
2000.0 323213.83999999997
2000.0000000000005 338478.55999999994
2000.0 1225060.3599999999
2000.0 1749163.8599999999
2000.0 1793069.0599999998
1999.9999999999995 1811837.0599999998
1999.9999999999998 1820321.4799999997
1999.9999999999995 1831485.2299999997
2000.0 2562465.2299999995
2000.0 10134550.829999998
2000.0 25569.60

2000.0000000009313 43442.0
2000.0000000009313 445968.95
2000.0000000009459 758637.0
2000.0000000009459 809245.8
2000.0000000009459 825722.01
2000.0000000009313 2361183.01
2000.0000000009313 8652480.23
2000.0000000009295 8730048.23
2000.0000000009295 8755860.23
2000.0000000009295 8765153.030000001
2000.0000000009295 8775703.430000002
2000.0000000009313 10097104.730000002
2000.0000000009313 38330979.45
2000.0000000009313 47623.25
2000.0000000009313 446085.64999999997
2000.0000000009313 773110.9299999999
2000.0000000009313 828770.9299999999
2000.0000000009304 846204.1299999999
2000.0000000009895 2375572.25
2000.0000000009313 9633293.71
2000.0000000009313 9713408.71
2000.0000000009313 9740602.15
2000.0000000009309 9749630.950000001
2000.0000000009309 9760382.950000001
2000.0000000009313 10907723.950000001
2000.0000000009313 37927791.7
2000.0000000009313 45413.799999999996
2000.0000000009313 399498.3
2000.0000000009313 765889.8200000001
2000.0000000009313 827300.8200000001
2000.000000000931

2000.0 77284.0
2000.0 810277.62
2000.0 2053534.46
2000.0 2191061.46
1999.9999999999995 2210267.2199999997
2000.0 8024904.18
2000.0 8151449.279999999
2000.0 8190050.4799999995
2000.0 8198699.47
2000.0 8212392.97
2000.0 9755928.969999999
2000.0 138456671.57
2000.0000000000018 76252.79999999999
2000.0 767641.8
2000.0 1984861.55
2000.0 2133693.05
2000.0000000000005 2151978.05
2000.0 8171079.55
2000.0 8302968.1899999995
2000.0 8346690.9399999995
2000.0000000000002 8355118.9399999995
2000.0 8369328.56
2000.0 9996328.559999999
2000.0 134697356.56
2000.0 74060.0
2000.0 829021.2000000001
2000.0 2032997.0
1999.9999999999964 2188822.03
1999.9999999999964 2206918.03
2000.0 8343181.029999999
2000.0 8488755.03
2000.0 8533701.229999999
2000.0 8542501.629999999
1999.9999999999995 8557633.329999998
2000.0 10180526.329999998
2000.0 135134988.82999998
2000.0 71306.64
2000.0 904256.6699999999
2000.0 2100848.92
2000.0 2262601.7199999997
1999.9999999999995 2281362.67
2000.0 8590365.49
2000.0 8745068.09
2000

2000.0 21542235.76
2000.0 21552549.44
1999.9999999999995 21569050.990000002
2000.0 227382931.98999998
2000.0 94608.0
2000.0 2350514.0
2000.0 3214300.1
2000.0 3268629.6
2000.0 20699047.95
2000.0 21123518.15
2000.0 21250349.09
2000.0 21260817.21
1999.9999999999995 21277271.060000002
2000.0 225253585.56
2000.0 93504.4
2000.0 2389773.1999999997
2000.0 3225333.1999999997
2000.0 3276879.2399999998
2000.0 20826341.24
2000.0 21228747.439999998
2000.0 21358707.439999998
2000.0 21368785.04
2000.0 21385347.54
2000.0 225405631.54
2000.0 95424.24
2000.0000000000582 2605603.64
2000.0000000000582 3557691.64
2000.0000000000582 3613869.3400000003
2000.0 21866656.09
2000.0 22282133.59
2000.0 22408069.59
2000.0000000000005 22418743.11
2000.0000000000005 22434643.11
2000.0 233723935.10999995
2000.0 95796.0
2000.0 2693016.0
2000.0 3631577.0
2000.0 3694562.0
2000.0 21531952.0
2000.0 21968621.74
2000.0 22092152.58
2000.0 22102656.58
2000.0000000000005 22117761.58
2000.0 234598751.48000002
2000.0 97861.319999

2000.0000000000036 160384.0
2000.0000000001164 3599284.0
2000.0000000001164 5187744.5
2000.0000000001164 5278078.25
2000.0 39120696.3
2000.0 39885980.4
2000.0 40012400.4
2000.0000000000005 40023380.199999996
2000.000000000001 40041921.099999994
2000.0 404562793.9
1999.9999999999964 158633.8
2000.0 3682338.2499999995
2000.0 5356450.75
2000.0 5448007.48
2000.0 34689227.18
2000.0 35491255.18
2000.0 35607144.18
1999.999999999999 35618274.78
1999.9999999999995 35638707.28
2000.0 386897833.68000007
2000.0 191580.09
2000.0 3559704.51
2000.0 5177161.5
2000.0 5270172.5
2000.0 41147040.0
2000.0 41912056.98
2000.0 42027515.86
1999.9999999999995 42039178.42
2000.0 42059681.870000005
2000.0 446061025.27000004
2000.0 205479.0
2000.0 3505533.6
2000.0 5027182.1
2000.0 5121370.1
2000.0 43742114.1
2000.0 44545849.35
2000.0 44664220.35
2000.0000000000005 44675857.95
2000.0 44697197.95
2000.0 460245535.0
2000.0 199208.16
2000.0 3341074.16
2000.0 5212174.16
2000.0 5307471.61
2000.0 39472056.61
2000.0 40248

2000.0 364609.0
2000.0 4981522.74
2000.0 8583276.74
2000.0 8740903.84
2000.0 60115563.72
2000.0 61539236.72
2000.0 61716711.12
1999.9999999999998 61726096.32
2000.0 61748300.88
2000.0 894837047.04
2000.0 423343.75
2000.0 4637332.2299999995
2000.0 7989310.24
2000.0000000000036 8150213.84
2000.0 60059223.64
2000.0 61505652.14
2000.0 61692765.480000004
2000.0 61702037.28
2000.0 61723306.08
2000.0 955664969.38
2000.0000000000146 532288.12
2000.0 4783803.670000001
2000.0 9299913.670000002
2000.0 9469048.96
2000.0 58653441.58
2000.0 60100981.94
2000.0 60291300.28
2000.0000000000002 60299550.28
2000.0 60321071.08
2000.0 944615812.08
2000.0 521776.62
2000.0 5382063.62
2000.0 5560162.62
2000.0 48523786.62
2000.0 49990711.919999994
2000.0 50184387.419999994
2000.0000000000005 50193022.419999994
2000.0 50214794.099999994
2000.0 967819470.1999999
2000.0000000000146 670387.2
2000.0 5984947.2
2000.0 6166689.2
2000.0 52102036.7
2000.0 53542440.7
2000.0 53731219.760000005
2000.0 53740840.56
2000.0 537

2000.0000000596046 1864544.1
2000.0000000596046 10988676.6
2000.0000000596046 11522572.32
2000.0000000596042 11534782.32
2000.0000000596037 11556641.82
2000.0000000596046 1696250588.4899998
2000.0000000596046 1951797.7
2000.0000000596046 12143471.299999999
2000.0000000596046 12666629.299999999
2000.0000000596046 12680535.02
2000.0000000596046 12703551.049999999
2000.0000000596046 1737343391.05
2000.0000000596046 2150561.6999999997
2000.0000000596046 10966420.05
2000.0000000596046 11490623.3
2000.000000059605 11505099.3
2000.0000000596046 11527443.3
2000.0000000596046 1904773904.9399998
2000.0000000596046 2119667.5
2000.0000000596046 11937538.0
2000.0000000596046 12454288.0
2000.000000059605 12468954.96
2000.0000000596056 12490225.65
2000.0000000596046 1819326123.99
2000.0000000596046 2060415.0
2000.0000000596046 12031483.260000002
2000.0000000596046 12554192.760000002
2000.0000000596046 12569019.110000001
2000.0000000596046 12590283.530000001
2000.0000000596046 1847999684.8100002
2000.

2000.0000002384186 13438425.0
2000.0000002384186 13471043.0
2000.0000002384186 5851300903.8
2000.0000002384186 13440217.0
2000.0000002384195 13473562.0
2000.0000002384186 5671856017.2699995
2000.0000002384186 12917329.0
2000.0000002384195 12950308.25
2000.0000002384186 5404852116.62
2000.0000002384186 12211242.0
2000.0000002384177 12242250.0
2000.0000002384186 5693112780.32
2000.0000002384186 12239311.8
2000.0000002384186 12269018.05
2000.0000002384186 5420225544.75
2000.0000002384186 11486202.0
2000.0000002384186 11514842.5
2000.0000002384186 5020973449.8
2000.0000002384186 11413156.0
2000.0000002384186 11440480.0
2000.0000002384186 5354270032.5
2000.0000002384186 11914529.370000001
2000.0000002384186 11941839.370000001
2000.0000002384186 4874063117.7699995
2000.0000002384186 12323834.600000001
2000.0000002384186 12354077.600000001
2000.0000002384186 4913281812.18
2000.0000002384186 12318403.5
2000.0000002384186 12349105.5
2000.0000002384186 5369821925.400001
2000.0000002384186 142646